# INFO 4271 - Exercise 3 - Indexing

Issued: April 30, 2024

Due: May 6, 2024

Please submit this filled sheet via Ilias by the due date.

---

# 1. Skip Pointers
Skip pointers can be used to accelerate posting list intersection, allowing pointers to be moved either to the next sequential list position or to the position of the skip pointer if one is available.

a) Implement the `intersect\_skip()` function sketched below. Each time you would ordinarily increment a pointer by one, you can alternatively follow a skip pointer, if one is available at the position.

In [19]:
#Intersect two sorted posting lists that contain skip pointers
def intersect_skip(A, B, skip=True):
    result = []
    i, j = 0, 0
    incs = 0

    while i < len(A) and j < len(B):
        docID_A = A[i][0]
        docID_B = B[j][0]

        if skip:
            if i < len(A) and A[i][1] is not None:
                skip_index_A = A[i][1]
                if A[i][2] <= docID_B:
                    i = skip_index_A
                    continue

            if j < len(B) and B[j][1] is not None:
                skip_index_B = B[j][1]
                if B[j][2] <= docID_A:
                    j = skip_index_B
                    continue

        if docID_A == docID_B:
            result.append(docID_A)
            i += 1
            j += 1
            incs += 2
        elif docID_A < docID_B:
            i += 1
            incs += 1
        else:
            j += 1
            incs += 1

    print("Increments used: ", incs)
    return result

#Posting lists with skip pointers.
#Entries take the form [docID, index to skip to, docID at that index]
times_skip = [[2, 3, 16], [12, None, None], [15, None, None], [16, 6, 27], [17, None, None], [23, None, None], [27, None, None]]
square_skip = [[3, 2, 12], [8, None, None], [12, 4, 23], [19, None, None], [23, None, None]]

print(intersect_skip(times_skip, square_skip))
print(intersect_skip(times_skip, square_skip, skip=False))

Increments used:  9
[12, 23]
Increments used:  11
[12, 23]


b) How many pointer increment operations did you need to intersect the two posting lists with the given skip pointers? How many operations would it have been for the same lists without skip pointers?

With skipping 9 increments are used without skipping 11 are used.

# 2. Positional Indices
Positional indices include for each posting the exact positions at which the term can be found in the document. This information allows us to satisfy two previously impossible types of queries. 1) Phrase queries require terms to occur adjacently to one another in a specific order. 2) Range queries allow for more leeway between term positions, merely requiring the two terms to appear within a specified number of tokens.

Implement the `intersect\_range()` function sketched in the code base. Each time you would ordinarily have reported a match, you will now need to check whether the range requirement is satisfied. As an optional addition, think about making this range check efficient using some of the techniques discussed for general posting list intersection.

In [20]:
import numpy as np

#Intersect two sorted posting lists with document-internal proximity requirements.
def intersect_range(A, B, range):
    result = []
    i, j = 0, 0
    incs = 0

    while i < len(A) and j < len(B):
        docID_A = A[i][0]
        docID_B = B[j][0]

        if docID_A == docID_B:
            a = np.array(A[i][1])
            b = np.array(B[j][1])
            diff = np.abs(a - b.reshape(-1, 1))
            if diff.min() <= range:
                result.append(docID_A)
            i += 1
            j += 1
            incs += 2
        elif docID_A < docID_B:
            i += 1
            incs += 1
        else:
            j += 1
            incs += 1

    return result

#Posting lists with document-internal positional information.
#Entries take the form [docID, [pos1, pos2, ...]]
times_range = [[2, [15, 128]], [12, [6, 45, 89, 942]], [15, [13]], [16, [1276, 1500]], [17, [13, 89, 90]], [23, [17, 64]], [27, [456, 629]]]
square_range = [[3, [65, 90]], [8, [67, 94]], [12, [3]], [19, [18, 81, 1881]], [23, [63]]]

print(intersect_range(times_range, square_range, 3))

[12, 23]


# 3. Paper Pick
Don't forget to submit your paper pick at https://forms.gle/SFYUKxiMXZKbs5XCA.